<a href="https://colab.research.google.com/github/PalakDograCSE-AI2004/CVDL_Summer_Internship/blob/main/HAR_case_study2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#here we are doing 2nd case study with defining ie writing whole alexnet pre trained cnn model architecture ourself

In [ ]:
#import os
os.environ["KAGGLE_USERNAME"]="priyamalik630"
os.environ["KAGGLE_KEY"]="3e7ac7acd3510bbb936768912ac3dffd"
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

401 - Unauthorized


In [ ]:
#!unzip human-action-recognition-har-dataset.zip

unzip:  cannot find or open human-action-recognition-har-dataset.zip, human-action-recognition-har-dataset.zip.zip or human-action-recognition-har-dataset.zip.ZIP.


In [ ]:
# PIL
# opencv

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import torch
from torch import nn
from torchvision import transforms
from torch.utils import data
import torchvision.datasets as datasets

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

**Explore Dataset**

In [ ]:
train_path = "Human Action Recognition/train"
test_path = "Human Action Recognition/test"

In [ ]:
random_img = train_path + "/" + "Image_5.jpg"

In [ ]:
img = cv2.imread(random_img)

In [ ]:
type(img)


In [ ]:
img.shape

In [ ]:
plt.imshow(img)
plt.show()


In [ ]:
df = pd.read_csv("Human Action Recognition/Training_set.csv")


In [ ]:

df.head()

In [ ]:
pd.value_counts(df['label'])


In [ ]:
class_names = pd.value_counts(df['label']).index

In [ ]:
class_names = np.sort(class_names)
print(class_names)

In [ ]:
df.iloc[545]

In [ ]:

filenames = df['filename'].values

In [ ]:

filenames[:10]

In [ ]:
df['label'][0]


In [ ]:
# transform = transforms.Compose([transforms.ToTensor()])

def load_data(path, df):
  images_list = []
  labels_list = []
  for i in tqdm(range(len(filenames))):
    # concat train_path with image name
    img_path = path + "/" + filenames[i]
    # fetch image label from data frame of current image
    img_label = df['label'][i]
    # read image using opencv
    img = cv2.imread(img_path)
    # resize image because images might be of different dimensions
    # in order to maintain array, we have to resize all the images in same dimension
    # img = cv2.resize(img, (150,150))
    # img = transform(img)
    # img = img / 255.0
    # store images one by one in your list
    images_list.append(img)
    labels_list.append(img_label)

  images_arr = np.asarray(images_list)
  labels_arr = np.asarray(labels_list)

  return images_arr, labels_arr

In [ ]:

train_df = pd.read_csv("Human Action Recognition/Training_set.csv")
# test_df = pd.read_csv("Human Action Recognition/Testing_set.csv")
#we will only deal with train data

In [ ]:
# test_df.head()

In [ ]:
train_images, train_labels = load_data(train_path, train_df)

In [ ]:
train_images.shape

In [ ]:
plt.imshow(train_images[1670])
plt.show()


In [ ]:
train_labels[0]

In [ ]:
# Inherit Dataset class coming from data package
class Dataset(data.Dataset):
  def __init__(self, images, labels):
    self.transforms = transforms
    self.images = images
    self.labels = labels

  def __len__(self):
    return len(self.images)

  def __getitem__(self, index):
    # loading data - one image at a time
    X = self.images[index]
    X = cv2.resize(img,(227,227))
    y = self.labels[index]
    transform = transforms.Compose([transforms.ToTensor()])
    X = transform(X)
    # X = torch.tensor(X)
    # X = torch.cat((X,X,X),0)
    return X, y


In [ ]:
np.unique(train_labels, return_counts=True)


In [ ]:
label = LabelEncoder()
train_labels = label.fit_transform(train_labels)

In [ ]:
np.unique(train_labels, return_counts=True)

In [ ]:
# 75% - training and 25% - testing
x_train, x_test, y_train, y_test = train_test_split(train_images, train_labels, test_size=0.25)


In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
params = {"batch_size":32, "shuffle":True}

training_set = Dataset(x_train, y_train)
training_generator = data.DataLoader(training_set, **params)

test_set = Dataset(x_test, y_test)
test_generator = data.DataLoader(test_set, **params)


In [ ]:
# Normalization
# x_train, x_test = x_train / 255.0, x_test / 255.0


In [ ]:

class AlexNet(nn.Module):
  def __init__(self, num_classes=15):
    super().__init__()
    self.cnn_blocks = nn.Sequential(
        nn.Conv2d(3, 96, kernel_size=11, stride=4),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.Conv2d(96, 256, kernel_size=3, stride=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.Conv2d(256, 384, kernel_size=3, stride=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(384, 384, kernel_size=3, stride=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(384, 256, kernel_size=3, stride=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2),
    )
    self.fcn_blocks = nn.Sequential(
        nn.Linear(1024, 4096),
        nn.ReLU(inplace=True),
        nn.Linear(4096, 4096),
        nn.ReLU(inplace=True),
        nn.Linear(4096, num_classes),
    )

  def forward(self, x):
    x = self.cnn_blocks(x)
    x = torch.flatten(x,1)
    x = self.fcn_blocks(x)
    return x

In [ ]:
model = AlexNet()


model

In [ ]:
def accuracy(y_true, y_pred):
  correct_classification = torch.eq(y_true, y_pred).sum().item()
  acc = (correct_classification / len(y_pred)) * 100
  return acc


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

In [ ]:
type(x_train)

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Evaluation metric
def accuracy(y_true, y_pred):
  # y_true = 1, y_pred = 1
  # y_true = 0, y_pred = 0
  correct_classification = torch.eq(y_true, y_pred).sum().item()
  acc = (correct_classification / len(y_pred)) * 100
  return acc

def train_step(epoch, model, data, loss_fn, optimizer):
  train_loss, train_acc = 0,0
  model.to(device)

  for batch, (X, y) in enumerate(data):
    X,y = X.to(device), y.to(device)

    # Feedforward - it calls forward method inside Model Class
    y_pred = model(X)
    # Calculate loss
    loss = loss_fn(y_pred, y)
    train_loss += loss
    train_acc += accuracy(y, y_pred.argmax(dim=1))

    # Backpropagate
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  train_loss /= len(data)
  train_acc /= len(data)
  train_acc_history.append(train_acc)
  train_loss_history.append(train_loss)
  print(f"Epoch : {epoch} | Train Loss : {train_loss:.3f} |  Train Acc : {train_acc:.3f}")


def test_step(epoch, model, data, loss_fn, optimizer):
  test_loss, test_acc = 0,0
  model.to(device)
  model.to(eval)

  with torch.inference_mode():
    for batch, (X, y) in enumerate(data):
      X,y = X.to(device), y.to(device)

      # Feedforward - it calls forward method inside Model Class
      y_pred = model(X)
      # Calculate loss
      loss = loss_fn(y_pred, y)
      test_loss += loss
      test_acc += accuracy(y, y_pred.argmax(dim=1))

    test_loss /= len(data)
    test_acc /= len(data)
    print(f"Epoch : {epoch} | Test Loss : {test_loss:.3f} |  Test Acc : {test_acc:.3f}")

In [ ]:
model = AlexNet().to(device)
model.to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)
loss_function = nn.CrossEntropyLoss()

train_acc_history = []
train_loss_history = []

epochs = 20

for epoch in tqdm(range(epochs)):
  train_step(epoch, model, training_generator, loss_function, optimizer)
  # test_step(epoch, model, test_generator, loss_function, optimizer)